In [1]:
import os 
os.chdir("../..")

In [2]:
import numpy as np
#from detectron2.structures import BoxMode
import cv2
import matplotlib.pyplot as plt


from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from src.utils.dataset import register_detectron2_multipart_datasets
from src.utils.training import get_splits_for_multipart_dataset

import tqdm

In [3]:
!ls data/training/RGBN

lubsko		swiebodzin  torzym		       zagan
sulechow_slawa	szprotawa   train_val_test_splits.pkl


In [4]:
base_path = "data/training/RGBN/"
train_val_maps = ["szprotawa", "swiebodzin", "lubsko", "zagan", "torzym"]
test_maps = ["sulechow_slawa"]

In [5]:
splits = get_splits_for_multipart_dataset(base_path, train_val_maps, test_maps)

In [6]:
register_detectron2_multipart_datasets("sick_trees", 
                                       {m: base_path + m for m in train_val_maps},
                                       splits, min_bbox_area=200)

In [7]:
meta = MetadataCatalog.get('sick_trees_train')

In [8]:
%%time
train_dataset_dicts = DatasetCatalog.get("sick_trees_train")
len(train_dataset_dicts)

CPU times: user 12.5 s, sys: 15.3 ms, total: 12.5 s
Wall time: 12.5 s


22069

In [9]:
%%time
val_dataset_dicts = DatasetCatalog.get("sick_trees_val")
len(val_dataset_dicts)

CPU times: user 3.73 s, sys: 4.11 ms, total: 3.73 s
Wall time: 3.73 s


5504

In [10]:
means = {c: [] for c in "rgbn"}
stds = {c: [] for c in "rgbn"}

for d in tqdm.notebook.tqdm(train_dataset_dicts):
    im = cv2.imread(d["file_name"], cv2.IMREAD_UNCHANGED)
    for i, c in enumerate("rgbn"):
        means[c].append(im[:,:,i].mean())
        stds[c].append(im[:,:,i].std())
    
# print("mean of means: ", np.mean(ndvi_means))
# print("mean of stds:  ", np.mean(ndvi_stds))
# print("std of means:  ", np.std(ndvi_means))
for c in "rgbn":
    print(c, "mean: ", np.mean(means[c]))
    print(c, "std: ", np.mean(stds[c]))


r mean:  68.60176277048289
r std:  25.05072945688351
g mean:  90.37255792238817
g std:  22.96708310183466
b mean:  99.04053918695983
b std:  13.27640256812483
n mean:  187.12327778336686
n std:  10.55706656507702


In [11]:
# print("std of stds:  ", np.std(ndvi_stds))

std of stds:   6.853606558543485


In [11]:
[np.mean(means[c]) for c in "rgbn"]

[68.60176277048289, 90.37255792238817, 99.04053918695983, 187.12327778336686]

In [12]:
[np.mean(stds[c]) for c in "rgbn"]

[25.05072945688351, 22.96708310183466, 13.27640256812483, 10.55706656507702]